<a href="https://colab.research.google.com/github/git-ginwook/InsightToInterface/blob/movies_bert/InsightToInterface.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Movie Classification

## Dataset


[IMDB Dataset of 50K Movie Reviews](https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews/code)

- connect dataset directly from Kaggle

In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("lakshmi25npathi/imdb-dataset-of-50k-movie-reviews")

print("Path to dataset files:", path)

100%|██████████| 25.7M/25.7M [00:00<00:00, 188MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews/versions/1


In [2]:
import pandas as pd

reviews_df = pd.read_csv(path + "/IMDB Dataset.csv")
reviews_df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


from matplotlib import pyplot as plt
import seaborn as sns
_df_0.groupby('review').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
_df_1.groupby('sentiment').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
import pandas as pd
plt.subplots(figsize=(8, 8))
df_2dhist = pd.DataFrame({
    x_label: grp['sentiment'].value_counts()
    for x_label, grp in _df_2.groupby('review')
})
sns.heatmap(df_2dhist, cmap='viridis')
plt.xlabel('review')
_ = plt.ylabel('sentiment')

## GPU Device Setup

1. Verify A100 GPU set up

In [3]:
import torch

# Check if GPU is available
if torch.cuda.is_available():
    device = torch.device("cuda")  # Automatically selects the first available GPU
    gpu_name = torch.cuda.get_device_name(0)

    # Check if it's an A100 GPU
    if "A100" in gpu_name:
        print(f"Successfully set up {gpu_name}!")
    else:
        print(f"GPU assigned: {gpu_name}. Note: It's not an A100 GPU.")
else:
    device = torch.device("cpu")
    print("GPU not available. Using CPU.")

# Print CUDA version
print(f"CUDA Version: {torch.version.cuda}")
print(f"PyTorch Version: {torch.__version__}")


Successfully set up NVIDIA A100-SXM4-40GB!
CUDA Version: 12.1
PyTorch Version: 2.5.1+cu121


2. Verify GPU performance

In [4]:
import time

# Dummy tensor operation to benchmark GPU
device = torch.device("cuda")
size = 10000

# Create random tensors
a = torch.randn(size, size, device=device)
b = torch.randn(size, size, device=device)

# Time matrix multiplication on GPU
start = time.time()
c = torch.matmul(a, b)
torch.cuda.synchronize()  # Wait for GPU to finish
end = time.time()

print(f"Time for matrix multiplication on GPU: {end - start:.4f} seconds")


Time for matrix multiplication on GPU: 0.1657 seconds


## Models


[Customer Segmentation using LLMs: Advanced Clustering Techniques for Effective Targeting](https://ai.plainenglish.io/customer-segmentation-using-llms-advanced-clustering-techniques-for-effective-targeting-493116116ab6)

1. BERT for Text Embedding -> review sentiments (e.g., + or -)

In [7]:
model.device

device(type='cuda', index=0)

In [11]:
from transformers import BertTokenizer, BertModel
import torch

from sklearn.cluster import KMeans

# Load the pre-trained BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased').to(device) # use GPU

# Sample customer reviews
customer_reviews = reviews_df["review"][:1000].to_list()

# Tokenize and encode the reviews
# inputs = tokenizer(customer_reviews, return_tensors='pt', padding=True, truncation=True)
inputs = tokenizer(customer_reviews, return_tensors='pt', padding=True, truncation=True)
inputs.to(device) # load data to GPU

# Get the embeddings from the BERT model
with torch.no_grad():
    outputs = model(**inputs)
    embeddings = outputs.last_hidden_state.mean(dim=1)

# Convert embeddings to a numpy array for clustering
embeddings = embeddings.cpu().numpy() # bring back output from GPU to CPU

# Perform K-Means clustering
kmeans = KMeans(n_clusters=2)
labels = kmeans.fit_predict(embeddings)

# Add the results to a DataFrame for better visualization
df = pd.DataFrame({
    'Review': customer_reviews,
    'Cluster': labels
})

# Show the clusters
print(df)

                                                Review  Cluster
0    One of the other reviewers has mentioned that ...        0
1    A wonderful little production. <br /><br />The...        1
2    I thought this was a wonderful way to spend ti...        1
3    Basically there's a family where a little boy ...        1
4    Petter Mattei's "Love in the Time of Money" is...        0
..                                                 ...      ...
995  Nothing is sacred. Just ask Ernie Fosselius. T...        1
996  I hated it. I hate self-aware pretentious inan...        1
997  I usually try to be professional and construct...        0
998  If you like me is going to see this in a film ...        1
999  This is like a zoology textbook, given that it...        0

[1000 rows x 2 columns]


2. Latent Dirichlet Allocation (LDA) -> review themes (e.g., price, quality) - TBD

In [1]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

# Sample Dataset
documents = [
    "Machine learning is fascinating.",
    "Artificial intelligence and deep learning are branches of machine learning.",
    "I like icecream.",
    "AI and machine learning are transforming industries.",
    "Blueberry is delicious."
]

# Step 1: Text Preprocessing (e.g., Tokenization, Stopword Removal, Vectorization)
vectorizer = CountVectorizer(
    max_df=0.95,  # Ignore terms with a document frequency > 95%
    min_df=2,     # Ignore terms with a document frequency < 2
    stop_words='english'  # Remove common stopwords
)
X = vectorizer.fit_transform(documents)

# Step 2: Apply LDA
n_topics = 2  # Specify the number of topics
lda_model = LatentDirichletAllocation(
    n_components=n_topics,   # Number of topics
    max_iter=10,             # Maximum number of iterations
    learning_method='batch', # Batch or online learning
    random_state=42          # Random seed for reproducibility
)
lda_model.fit(X)

# Step 3: Display Topics
def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print(f"Topic {topic_idx + 1}:")
        print(" ".join([feature_names[i] for i in topic.argsort()[:-no_top_words - 1:-1]]))

no_top_words = 5  # Number of top words to display per topic
feature_names = vectorizer.get_feature_names_out()
display_topics(lda_model, feature_names, no_top_words)

# Step 4: Topic Distribution for Documents
doc_topic_distribution = lda_model.transform(X)
print("\nDocument-Topic Distributions:")
print(doc_topic_distribution)


Topic 1:
learning machine
Topic 2:
machine learning

Document-Topic Distributions:
[[0.72330431 0.27669569]
 [0.81495224 0.18504776]
 [0.5        0.5       ]
 [0.72330431 0.27669569]
 [0.5        0.5       ]]
